## Notebook Cientifico de Datos
### Valentina Espitia
### Miguel García

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
from itertools import combinations
from xgboost import XGBClassifier
import re
from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, evaluate_model, finalize_model, predict_model, save_model, load_model
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

## Carga de las bases de datos

In [27]:
path = "C:/Users/MCS/OneDrive - Universidad Santo Tomás/Inteligencia Artificial/Codigos Propios/PARCIAL 1 CORTE/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
train = train.drop(columns = ["id"])
id = test["id"]
test = test.drop(columns = ["id"])
categ = ['Marital status','Application mode', 'Application order', 'Daytime/evening attendance', 'Nacionality',
         "Mother's occupation", "Father's occupation", 'Displaced', 'Educational special needs', 
         'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International', "Course"]
for k in categ:
  train[k] = train[k].astype("O")
  test[k] = test[k].astype("O")
for p in categ:
    train[p] = "C" + train[p].astype("str")
    test[p] = "C" + test[p].astype("str")

## Modelo sin ingieneria de variables

In [28]:
formato = pd.DataFrame({'Variable': list(train.columns), 'Formato': train.dtypes })
categoricas = list(formato.loc[formato["Formato"]=="O","Variable"])
categoricas = [x for x in categoricas if "Target" not in x]
numericas = list(formato.loc[formato["Formato"]!="O","Variable"])
train_num = train.get(numericas)
train_cat = train.get(categoricas)

In [34]:
%%time
exp_clf101 = setup(data = train, target='Target', session_id = 123, train_size = 0.7,
                   numeric_features = numericas, 
                   categorical_features = categoricas, fix_imbalance=False)
modelo1 = create_model("xgboost", verbose = False)
param_grid_bayesian = {'n_estimators': [50,100,200], 'max_depth': [3,5,7], 'min_child_samples': [50,150,200]}
tuned_modelo1 = tune_model(modelo1, custom_grid = param_grid_bayesian,
                          search_library = 'scikit-optimize', search_algorithm = 'bayesian', 
                           fold = 5)
modelo_final1 = finalize_model(tuned_modelo1)

,Description,Value
0,Session id,123
1,Target,Target
2,Target type,Multiclass
3,Target mapping,"Dropout: 0, Enrolled: 1, Graduate: 2"
4,Original data shape,"(76518, 37)"
5,Transformed data shape,"(76518, 99)"
6,Transformed train set shape,"(53562, 99)"
7,Transformed test set shape,"(22956, 99)"
8,Numeric features,21
9,Categorical features,15


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8308,0.9425,0.8308,0.8293,0.8289,0.7274,0.7287,69.3070
lightgbm,Light Gradient Boosting Machine,0.8299,0.9434,0.8299,0.8295,0.8284,0.7261,0.7275,69.7580
gbc,Gradient Boosting Classifier,0.8280,0.0000,0.8280,0.8272,0.8262,0.7228,0.7244,363.9810
rf,Random Forest Classifier,0.8261,0.9380,0.8261,0.8243,0.8236,0.7192,0.7210,23.3250
ada,Ada Boost Classifier,0.8208,0.0000,0.8208,0.8214,0.8194,0.7114,0.7131,59.7710
lr,Logistic Regression,0.8207,0.0000,0.8207,0.8166,0.8167,0.7096,0.7117,107.3020
et,Extra Trees Classifier,0.8202,0.9353,0.8202,0.8179,0.8170,0.7090,0.7113,74.0610
lda,Linear Discriminant Analysis,0.8100,0.0000,0.8100,0.8144,0.8087,0.6931,0.6967,28.2310
ridge,Ridge Classifier,0.8008,0.0000,0.8008,0.7961,0.7883,0.6696,0.6806,4.3930
svm,SVM - Linear Kernel,0.7688,0.0000,0.7688,0.7857,0.7531,0.6235,0.6447,26.2190


CPU times: total: 2min 18s
Wall time: 2h 29min 12s


In [53]:
def precision(tuned_modelo, exp_clf101):
    predicciones_train = predict_model(tuned_modelo)
    predicciones_test = predict_model(tuned_modelo, data = exp_clf101.get_config('X_train'))
    
    y_train = exp_clf101.get_config('y_train')
    y_test = exp_clf101.get_config('y_test')
    
    u1 = accuracy_score(y_test,predicciones_train["prediction_label"])
    u2 = accuracy_score(y_train,predicciones_test["prediction_label"])
    return u1, u2

In [54]:
u1_1, u2_1 = precision(tuned_modelo1, exp_clf101)
print(u1_1, u2_1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8325,0.9446,0.8325,0.8315,0.8309,0.7304,0.7316


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8301,0.9439,0.8301,0.8299,0.8288,0.7267,0.7280


In [56]:
Predicciones1 = predict_model(modelo_final1, data = test)
resultados1 = pd.DataFrame({'id': id, 'Target': Predicciones1["prediction_label"]})
path = "C:/Users/MCS/OneDrive - Universidad Santo Tomás/Inteligencia Artificial/Codigos Propios/PARCIAL 1 CORTE/"
test1 = test = pd.read_csv(path + "test.csv")
id = test1["id"]
resultados1.to_csv(path + 'subbmision1_2IA.csv', index=False,sep=",")